In [1]:
import openai

import tenacity
import time

In [2]:
from bs4 import BeautifulSoup
import random

In [3]:
import json

In [4]:
# Replace with your OpenAI API key
openai.api_key = "<REDACTED>"

## Original Survey

In [5]:
survey_text = """
<h1>How do you think young adults today compare with their parents' generation on each of the following?</h1>
  <p>For the following questions, is this action easier, harder, or the same for young adults today compared to their
    parents' generation?</p>
  <crowd-form><form method="" action="">
    <div id="container">
    <div id="0ad80e72-02e7-4cfc-a298-cc1568f665ff&quot;">
        <div class="QuestionContent">
          <p>Buying a home</p>
        </div>
        <div>
          <input type="radio" id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Easier" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" value="'Easier" required="">
          <label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" value="'Harder" required="">
          <label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Same" name="0ad80e72-02e7-4cfc-a298-cc1568f665ff" value="'Same" required="">
          <label for="0ad80e72-02e7-4cfc-a298-cc1568f665ff-'Same">About the same</label>
        </div>
      </div><div id="c799476d-db52-4dfe-a5f2-187e3894d280&quot;">
        <div class="QuestionContent">
          <p>Staying in touch with family and friends</p>
        </div>
        <div>
          <input type="radio" id="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier" name="c799476d-db52-4dfe-a5f2-187e3894d280" value="'Easier" required="">
          <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder" name="c799476d-db52-4dfe-a5f2-187e3894d280" value="'Harder" required="">
          <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="c799476d-db52-4dfe-a5f2-187e3894d280-'Same" name="c799476d-db52-4dfe-a5f2-187e3894d280" value="'Same" required="">
          <label for="c799476d-db52-4dfe-a5f2-187e3894d280-'Same">About the same</label>
        </div>
      </div><div id="40b468aa-4354-4679-a297-068cc7d8f897&quot;">
        <div class="QuestionContent">
          <p>Paying for college</p>
        </div>
        <div>
          <input type="radio" id="40b468aa-4354-4679-a297-068cc7d8f897-'Easier" name="40b468aa-4354-4679-a297-068cc7d8f897" value="'Easier" required="">
          <label for="40b468aa-4354-4679-a297-068cc7d8f897-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="40b468aa-4354-4679-a297-068cc7d8f897-'Harder" name="40b468aa-4354-4679-a297-068cc7d8f897" value="'Harder" required="">
          <label for="40b468aa-4354-4679-a297-068cc7d8f897-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="40b468aa-4354-4679-a297-068cc7d8f897-'Same" name="40b468aa-4354-4679-a297-068cc7d8f897" value="'Same" required="">
          <label for="40b468aa-4354-4679-a297-068cc7d8f897-'Same">About the same</label>
        </div>
      </div><div id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d&quot;">
        <div class="QuestionContent">
          <p>Finding a job</p>
        </div>
        <div>
          <input type="radio" id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Easier" name="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d" value="'Easier" required="">
          <label for="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Harder" name="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d" value="'Harder" required="">
          <label for="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Same" name="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d" value="'Same" required="">
          <label for="2b6374ec-59d9-4ef2-9d8d-3eb910870b4d-'Same">About the same</label>
        </div>
      </div><div id="8f511d04-bcba-43a6-ac5f-eccdf5f049af&quot;">
        <div class="QuestionContent">
          <p>Getting into college</p>
        </div>
        <div>
          <input type="radio" id="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Easier" name="8f511d04-bcba-43a6-ac5f-eccdf5f049af" value="'Easier" required="">
          <label for="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Harder" name="8f511d04-bcba-43a6-ac5f-eccdf5f049af" value="'Harder" required="">
          <label for="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Same" name="8f511d04-bcba-43a6-ac5f-eccdf5f049af" value="'Same" required="">
          <label for="8f511d04-bcba-43a6-ac5f-eccdf5f049af-'Same">About the same</label>
        </div>
      </div><div id="d73751f3-b779-4351-914b-8605dd5c0bfc&quot;">
        <div class="QuestionContent" style="margin-bottom: -2em;">
          <p>What is your age?</p>
        </div>
        <crowd-input name="d73751f3-b779-4351-914b-8605dd5c0bfc" label="What is your age?" allowed-pattern="^\d*$" error-message="Please enter ONLY numbers" auto-validate="" required="" tabindex="0" aria-disabled="false">
        </crowd-input>
      </div><div id="f73efca0-d7fb-4fb6-9527-884cac8aac41&quot;">
        <div class="QuestionContent">
          <p>Finding a spouse or partner</p>
        </div>
        <div>
          <input type="radio" id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Easier" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" value="'Easier" required="">
          <label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Harder" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" value="'Harder" required="">
          <label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Same" name="f73efca0-d7fb-4fb6-9527-884cac8aac41" value="'Same" required="">
          <label for="f73efca0-d7fb-4fb6-9527-884cac8aac41-'Same">About the same</label>
        </div>
      </div><div id="727d4914-85d7-4e5d-8024-0d5af55027ac&quot;">
        <div class="QuestionContent">
          <p>Saving for the future</p>
        </div>
        <div>
          <input type="radio" id="727d4914-85d7-4e5d-8024-0d5af55027ac-'Easier" name="727d4914-85d7-4e5d-8024-0d5af55027ac" value="'Easier" required="">
          <label for="727d4914-85d7-4e5d-8024-0d5af55027ac-'Easier">Easier</label>
        </div>
        <div>
          <input type="radio" id="727d4914-85d7-4e5d-8024-0d5af55027ac-'Harder" name="727d4914-85d7-4e5d-8024-0d5af55027ac" value="'Harder" required="">
          <label for="727d4914-85d7-4e5d-8024-0d5af55027ac-'Harder">Harder</label>
        </div>
        <div>
          <input type="radio" id="727d4914-85d7-4e5d-8024-0d5af55027ac-'Same" name="727d4914-85d7-4e5d-8024-0d5af55027ac" value="'Same" required="">
          <label for="727d4914-85d7-4e5d-8024-0d5af55027ac-'Same">About the same</label>
        </div>
      </div><div id="1b56e8be-127c-47d3-9864-31fceb586234&quot;">
        <div class="QuestionContent">
          <p>Do you currently live in the United States?</p>
        </div>
        <div>
          <input type="radio" id="1b56e8be-127c-47d3-9864-31fceb586234-'Yes" name="1b56e8be-127c-47d3-9864-31fceb586234" value="'Yes" required="">
          <label for="1b56e8be-127c-47d3-9864-31fceb586234-'Yes">Yes</label>
        </div>
        <div>
          <input type="radio" id="1b56e8be-127c-47d3-9864-31fceb586234-'No" name="1b56e8be-127c-47d3-9864-31fceb586234" value="'No" required="">
          <label for="1b56e8be-127c-47d3-9864-31fceb586234-'No">No</label>
        </div>
      </div></div>
  <crowd-button form-action="submit" variant="primary" data-testid="crowd-submit">Submit</crowd-button></form></crowd-form>
  """

## Prompts

In [6]:
instruction = """I want you to act as an average Amazon Mechanical Turk (mturk) survey taker living in the US.
I will provide you with a HTML survey and you will generate a reasonable response to the survey.
You should respond with a JSON object to indicate your survey response, where the keys are the names in the webform, and the values are the values in the webform.
It is VERY IMPORTANT that you respond in valid JSON ONLY. Nothing before or after. Make sure to escape all strings
"""

instruction

'I want you to act as an average Amazon Mechanical Turk (mturk) survey taker living in the US.\nI will provide you with a HTML survey and you will generate a reasonable response to the survey.\nYou should respond with a JSON object to indicate your survey response, where the keys are the names in the webform, and the values are the values in the webform.\nIt is VERY IMPORTANT that you respond in valid JSON ONLY. Nothing before or after. Make sure to escape all strings\n'

In [7]:
sample_survey = """<h1>Food Survey</h1><form action="submit.php" method="post"><h2>Favorite Cuisine:</h2><label><input type="radio" name="favorite_cuisine" value="italian"> Italian</label><label><input type="radio" name="favorite_cuisine" value="mexican"> Mexican</label><label><input type="radio" name="favorite_cuisine" value="indian"> Indian</label><label><input type="radio" name="favorite_cuisine" value="chinese"> Chinese</label><label for="comments">Please share your thoughts:</label><textarea id="comments" name="comments"></textarea><input type="submit" value="Submit"></form>"""
sample_response = """{"favorite_cuisine": "italian","comments": "I love pizza!"}"""

## Running

In [8]:
def randomize_survey():
    soup = BeautifulSoup(survey_text, 'html.parser')
    
    # Extract questions
    elem_container = soup.find('div', id='container')
    
    # Shuffle questions
    questions = elem_container.find_all(recursive=False)
    random.shuffle(questions)
    
    # For each question, shuffle the options
    for question in questions:
        question_body = question.find_all(recursive=False)
    
        # shuffle options and not the first div .QuestionContent
        options = question_body[1:]
        random.shuffle(options)
        question_body[1:] = options
    
        question.extend(question_body)
    
    # Reconstruct the HTML string with shuffled questions and options
    shuffled_survey_text = str(soup).replace('\n', '')
    
    return shuffled_survey_text

In [14]:
# Define the function to make an API request
@tenacity.retry(
    wait=tenacity.wait_exponential(multiplier=1, max=60),  # Exponential backoff with max delay of 60 seconds
    stop=tenacity.stop_after_attempt(3),  # Retry 3 times at most
    retry=tenacity.retry_if_exception_type(openai.error.OpenAIError),  # Retry on OpenAI errors
)
def make_openai_request():
    shuffled_survey_text = randomize_survey()
    response = openai.ChatCompletion.create(
        model="gpt-4", 
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": "The HTML survey is given below:\n" + sample_survey},
            {"role": "assistant", "content": sample_response},
            {"role": "user", "content": "The HTML survey is given below:\n" + shuffled_survey_text},
        ]
    )
    return response

In [12]:
# Open the files for saving assistance responses and API responses
assistance_response_file = "assistance_responses.jsonl"
api_response_file = "raw_api_responses.jsonl"
error_response_file = "error_responses.jsonl"

assistance_responses = []

In [25]:
NUM_RESPONSE = 93

with open(assistance_response_file, "a") as assistance_output, open(api_response_file, "a") as api_output, open(error_response_file, "a") as error_output:
    try:
        for index in range(NUM_RESPONSE):
            print(index)
            # shuffle and send request
            api_response = make_openai_request()

            # save raw JSON request to api_response_file
            json.dump(api_response.to_dict_recursive(), api_output)
            api_output.write('\n')

            # try parsing the response into JSON and write accordingly
            response_text = api_response.choices[0].message.content.strip()
            try:
                # Attempt to decode response_text as JSON
                response_data = json.loads(response_text)
                print(response_data)
                assistance_responses.append(response_data)

                # Save to assistance_response_file
                json.dump(response_data, assistance_output)
                assistance_output.write('\n')
            except json.JSONDecodeError:
                # If decoding as JSON fails, save to error_response_file
                response_text = response_text.replace("\n", " ")
                error_output.write(response_text + '\n')

            # write to disc now
            assistance_output.flush()
            api_output.flush()
            error_output.flush()
            
            time.sleep(1)  # Optional: Add a delay between requests to avoid rate limits

    except tenacity.RetryError:
        print("Max retries reached. Exiting...")

0
{'0ad80e72-02e7-4cfc-a298-cc1568f665ff': "'Harder'", 'c799476d-db52-4dfe-a5f2-187e3894d280': "'Easier'", '40b468aa-4354-4679-a297-068cc7d8f897': "'Harder'", '2b6374ec-59d9-4ef2-9d8d-3eb910870b4d': "'Harder'", '8f511d04-bcba-43a6-ac5f-eccdf5f049af': "'Harder'", 'd73751f3-b779-4351-914b-8605dd5c0bfc': '30', 'f73efca0-d7fb-4fb6-9527-884cac8aac41': "'About the same'", '727d4914-85d7-4e5d-8024-0d5af55027ac': "'Harder'", '1b56e8be-127c-47d3-9864-31fceb586234': "'Yes'"}
1
{'0ad80e72-02e7-4cfc-a298-cc1568f665ff': 'Harder', 'c799476d-db52-4dfe-a5f2-187e3894d280': 'Easier', '40b468aa-4354-4679-a297-068cc7d8f897': 'Harder', '2b6374ec-59d9-4ef2-9d8d-3eb910870b4d': 'Harder', '8f511d04-bcba-43a6-ac5f-eccdf5f049af': 'Harder', 'd73751f3-b779-4351-914b-8605dd5c0bfc': '35', 'f73efca0-d7fb-4fb6-9527-884cac8aac41': 'About the same', '727d4914-85d7-4e5d-8024-0d5af55027ac': 'Harder', '1b56e8be-127c-47d3-9864-31fceb586234': 'Yes'}
2
{'0ad80e72-02e7-4cfc-a298-cc1568f665ff': 'Harder', 'c799476d-db52-4dfe-a5f

In [24]:
print(len(assistance_responses))
assistance_responses

7


[{'0ad80e72-02e7-4cfc-a298-cc1568f665ff': "'Harder'",
  'c799476d-db52-4dfe-a5f2-187e3894d280': "'Easier'",
  '40b468aa-4354-4679-a297-068cc7d8f897': "'Harder'",
  '2b6374ec-59d9-4ef2-9d8d-3eb910870b4d': "'Harder'",
  '8f511d04-bcba-43a6-ac5f-eccdf5f049af': "'Easier'",
  'd73751f3-b779-4351-914b-8605dd5c0bfc': '30',
  'f73efca0-d7fb-4fb6-9527-884cac8aac41': "'Same'",
  '727d4914-85d7-4e5d-8024-0d5af55027ac': "'Same'",
  '1b56e8be-127c-47d3-9864-31fceb586234': "'Yes'"},
 {'0ad80e72-02e7-4cfc-a298-cc1568f665ff': 'Harder',
  'c799476d-db52-4dfe-a5f2-187e3894d280': 'Easier',
  '40b468aa-4354-4679-a297-068cc7d8f897': 'Harder',
  '2b6374ec-59d9-4ef2-9d8d-3eb910870b4d': 'Harder',
  '8f511d04-bcba-43a6-ac5f-eccdf5f049af': 'Easier',
  'd73751f3-b779-4351-914b-8605dd5c0bfc': '28',
  'f73efca0-d7fb-4fb6-9527-884cac8aac41': 'Easier',
  '727d4914-85d7-4e5d-8024-0d5af55027ac': 'Harder',
  '1b56e8be-127c-47d3-9864-31fceb586234': 'Yes'},
 {'0ad80e72-02e7-4cfc-a298-cc1568f665ff': 'Harder',
  'c799476d-

## Cleaning

In [30]:
import pandas as pd

In [31]:
dfResponses = pd.DataFrame.from_records(assistance_responses)

In [32]:
dfResponses

,0ad80e72-02e7-4cfc-a298-cc1568f665ff,c799476d-db52-4dfe-a5f2-187e3894d280,40b468aa-4354-4679-a297-068cc7d8f897,2b6374ec-59d9-4ef2-9d8d-3eb910870b4d,8f511d04-bcba-43a6-ac5f-eccdf5f049af,d73751f3-b779-4351-914b-8605dd5c0bfc,f73efca0-d7fb-4fb6-9527-884cac8aac41,727d4914-85d7-4e5d-8024-0d5af55027ac,1b56e8be-127c-47d3-9864-31fceb586234
0,'Harder','Easier','Harder','Harder','Easier',30,'Same','Same','Yes'
1,Harder,Easier,Harder,Harder,Easier,28,Easier,Harder,Yes
2,Harder,Easier,Harder,Harder,Harder,32,Easier,Harder,Yes
3,Harder,Easier,Harder,Easier,Same,28,Easier,Harder,Yes
4,Harder,Easier,Harder,Harder,Easier,35,About the Same,Harder,Yes
...,...,...,...,...,...,...,...,...,...
95,Harder,Easier,Harder,Easier,Harder,30,Easier,Harder,Yes
96,Harder,Easier,Harder,Same,Harder,30,Same,Harder,Yes
97,'Harder','Easier','Harder','Same','Harder',30,'Easier','Harder','Yes'
98,'Harder','Easier','Harder','Harder','Same',30,'Same','Harder','Yes'


In [35]:
df_gpt4 = dfResponses.rename(columns={
    '0ad80e72-02e7-4cfc-a298-cc1568f665ff': 'Buying home',
    'c799476d-db52-4dfe-a5f2-187e3894d280': 'Stay in touch',
    '40b468aa-4354-4679-a297-068cc7d8f897': 'Pay for college',
    '2b6374ec-59d9-4ef2-9d8d-3eb910870b4d': 'Finding job',
    '8f511d04-bcba-43a6-ac5f-eccdf5f049af': 'College Entry',
    'd73751f3-b779-4351-914b-8605dd5c0bfc': 'Age',
    'f73efca0-d7fb-4fb6-9527-884cac8aac41': 'Finding partner',
    '727d4914-85d7-4e5d-8024-0d5af55027ac': 'Saving for future',
    '1b56e8be-127c-47d3-9864-31fceb586234': 'Live in US',
}, errors="raise")
df_gpt4

,Buying home,Stay in touch,Pay for college,Finding job,College Entry,Age,Finding partner,Saving for future,Live in US
0,'Harder','Easier','Harder','Harder','Easier',30,'Same','Same','Yes'
1,Harder,Easier,Harder,Harder,Easier,28,Easier,Harder,Yes
2,Harder,Easier,Harder,Harder,Harder,32,Easier,Harder,Yes
3,Harder,Easier,Harder,Easier,Same,28,Easier,Harder,Yes
4,Harder,Easier,Harder,Harder,Easier,35,About the Same,Harder,Yes
...,...,...,...,...,...,...,...,...,...
95,Harder,Easier,Harder,Easier,Harder,30,Easier,Harder,Yes
96,Harder,Easier,Harder,Same,Harder,30,Same,Harder,Yes
97,'Harder','Easier','Harder','Same','Harder',30,'Easier','Harder','Yes'
98,'Harder','Easier','Harder','Harder','Same',30,'Same','Harder','Yes'


In [39]:
df_gpt4.to_csv('gpt4_raw.csv')

In [38]:
for c in df_gpt4.columns:
    print("---- %s ---" % c)
    print(df_gpt4[c].value_counts())
    print()

---- Buying home ---
Buying home
'Harder'    65
Harder      29
'Harder      6
Name: count, dtype: int64

---- Stay in touch ---
Stay in touch
'Easier'    65
Easier      29
'Easier      6
Name: count, dtype: int64

---- Pay for college ---
Pay for college
'Harder'    65
Harder      29
'Harder      6
Name: count, dtype: int64

---- Finding job ---
Finding job
'Harder'            36
'Same'              22
Harder              18
Easier               5
'About the same'     5
'Same                4
Same                 4
About the same       2
'Harder              2
'Easier'             2
Name: count, dtype: int64

---- College Entry ---
College Entry
'Harder'          43
Harder            13
'Easier'          12
Easier            10
'Same'            10
Same               3
'Easier            3
'Harder            3
About the same     2
'Same              1
Name: count, dtype: int64

---- Age ---
Age
35    34
30    29
32    10
28     9
29     4
27     3
25     3
34     3
33     2
35     1
26

## Data Cleaning

In [45]:
df_gpt4_cleaned = df_gpt4.replace({
    "'Harder'": "Harder",
    "'Harder": "Harder",
    "'Easier'": "Easier",
    "'Easier": "Easier",
    "'Same'": "Same",
    "'Same": "Same",
    "About the same": "Same",
    "'About the same'": "Same",
    "About the Same": "Same",
    "'Yes'": "Yes",
    "'Yes": "Yes",
})
df_gpt4_cleaned

,Buying home,Stay in touch,Pay for college,Finding job,College Entry,Age,Finding partner,Saving for future,Live in US
0,Harder,Easier,Harder,Harder,Easier,30,Same,Same,Yes
1,Harder,Easier,Harder,Harder,Easier,28,Easier,Harder,Yes
2,Harder,Easier,Harder,Harder,Harder,32,Easier,Harder,Yes
3,Harder,Easier,Harder,Easier,Same,28,Easier,Harder,Yes
4,Harder,Easier,Harder,Harder,Easier,35,Same,Harder,Yes
...,...,...,...,...,...,...,...,...,...
95,Harder,Easier,Harder,Easier,Harder,30,Easier,Harder,Yes
96,Harder,Easier,Harder,Same,Harder,30,Same,Harder,Yes
97,Harder,Easier,Harder,Same,Harder,30,Easier,Harder,Yes
98,Harder,Easier,Harder,Harder,Same,30,Same,Harder,Yes


In [46]:
for c in df_gpt4_cleaned.columns:
    print("---- %s ---" % c)
    print(df_gpt4_cleaned[c].value_counts())
    print()

---- Buying home ---
Buying home
Harder    100
Name: count, dtype: int64

---- Stay in touch ---
Stay in touch
Easier    100
Name: count, dtype: int64

---- Pay for college ---
Pay for college
Harder    100
Name: count, dtype: int64

---- Finding job ---
Finding job
Harder    56
Same      37
Easier     7
Name: count, dtype: int64

---- College Entry ---
College Entry
Harder    59
Easier    25
Same      16
Name: count, dtype: int64

---- Age ---
Age
35    34
30    29
32    10
28     9
29     4
27     3
25     3
34     3
33     2
35     1
26     1
31     1
Name: count, dtype: int64

---- Finding partner ---
Finding partner
Same      62
Easier    28
Harder    10
Name: count, dtype: int64

---- Saving for future ---
Saving for future
Harder    93
Same       7
Name: count, dtype: int64

---- Live in US ---
Live in US
Yes    100
Name: count, dtype: int64



In [47]:
df_gpt4_cleaned.to_csv('gpt4_cleaned.csv')